## Importa funções para o pré-processamento

In [1]:
from pre_processamento import *

from dlisio import dlis
import pandas as pd
import numpy as np

# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

## Carrega os dados

In [2]:
nomes_arquivos = []     # Armazena os nomes dos arquivos .dlis
leituras_dlis = []      # Armazena as leituras dos arquivos .dlis
nomes_anp = []          # Armazena os nomes obtidos das leituras

for file in glob.glob(r'**/Dados-dlis' + "/*.dlis", recursive=True):
    # Salva o nome do arquivo
    nomes_arquivos.append(file)

    # Salva os dados da leitura
    leitura, *tail = dlis.load(f'{file}')
    leituras_dlis.append(leitura)

    # Salva o nome do poço
    nome = leitura.origins[0].well_name
    nomes_anp.append(nome)

In [3]:
print(nomes_arquivos)
print(nomes_anp)

['Dados-dlis\\1-brsa-551-se_brsa_raw.dlis', 'Dados-dlis\\1-brsa-574-se_brsa_raw.dlis', 'Dados-dlis\\1-brsa-595-se_brsa_raw-1.dlis', 'Dados-dlis\\1-brsa-605-se_brsa_raw.dlis', 'Dados-dlis\\1-brsa-659-se_brsa_raw.dlis']
['1-FSG-1-SE', '1-FSJQ-1-SE', '1-BRSA-595-SE', '1-BRSA-605-SE', '1-BRSA-659-SE']


## Cria dicionário para armazenar os dados e respectivos nomes

In [3]:
# Casa itens da lista 'nome_anp_abreviados' com os itens da lista 'leituras_dlis'
pares = zip(nomes_anp, leituras_dlis)

# Cria dicionário 'dli_dict'
dli_dict = dict(pares)
dli_dict

{'1-FSG-1-SE': LogicalFile(AIT_SONIC_TLD_MCFL_018PUP),
 '1-FSJQ-1-SE': LogicalFile(GEOLOAD.1),
 '1-BRSA-595-SE': LogicalFile(GEOLOAD.1),
 '1-BRSA-605-SE': LogicalFile(AIT_SONIC_TLD_MCFL_048PUC),
 '1-BRSA-659-SE': LogicalFile(GEOLOAD.1)}

## Separa TODAS as curvas presentes nos .dlis de cada poço

In [4]:
channels_dict = {}

for key, poco in dli_dict.items():
    channels_list = []
    for frame in poco.frames:
        channels = frame.channels
        channels_list.append([channel.name for channel in channels])
    channels_dict[key] = sum(channels_list, [])

## Concatena os arquivos CSV das curvas em um único arquivo

In [6]:
import os

pasta = "Curvas_CSV"

# Lista para armazenar os dataframes repectivos aos .csv
df_list = []

# Itera sobre a pasta Curvas_CSV
for arquivo in os.listdir(pasta):
    file_path = os.path.join(pasta, arquivo)

    # Lê o arquivo CSV como um DF
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatena os DFs ao longo do eixo das colunas
df_concat = pd.concat(df_list, axis=0, ignore_index=True)

# Salva o DF concatenado em um .csv
df_concat.to_csv("Curvas_CSV/curvas_pocos.csv", index=False)

print("Arquivo Curvas_CSV/curvas_pocos.csv criado com sucesso")

Arquivo Curvas_CSV/curvas_pocos.csv criado com sucesso


## Processa os 'frames' dos poços 1-BRSA-551-SE e 1-BRSA-605-SE

#### Quantidade de 'frames' dos poços

In [5]:
poco_551 = dli_dict['1-FSG-1-SE']
poco_605 = dli_dict['1-BRSA-605-SE']

print(poco_551.frames)
print(poco_605.frames)

[Frame(60B), Frame(10B), Frame(5B), Frame(30B), Frame(120B), Frame(180B), Frame(20B), Frame(1B), Frame(2B)]
[Frame(60B), Frame(10B), Frame(30B), Frame(20B)]


#### Cria um dicionário para armazenar os 'frames' 

In [6]:
frames_dict_551 = {}
frames_dict_605 = {}

cria_frames_dict(frames_dict_551, poco_551)
cria_frames_dict(frames_dict_605, poco_605)

#### Cria um 'dataframe' para cada 'frame'

In [8]:
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'RHOZ', 'DRHO', 'BSZ', 'BS', 'HCAL', 'CAL', 'CALI', 'DCALI', 'DCAL', 'PE', 'DT', 'DTC', 'ILD', 'RILD', 'IEL', 'AIT90', 'AHT90', 'RT90', 'AT90', 'AO90', 'RT', 'AF90', 'AHF90', 'AFH90', 'LLD', 'RLLD', 'HDRS', 'HLLD', 'LL7', 'RLL7']

dataframes_dict_551 = {}
dataframes_dict_605 = {}

cria_dataframes_dict(frames_dict_551, dataframes_dict_551, curvas_escolhidas)
cria_dataframes_dict(frames_dict_605, dataframes_dict_605, curvas_escolhidas)

#### Move a vírgula uma casa para a esquerda em todos os valores das colunas TDEP

In [9]:
for value in dataframes_dict_551.values():
    value["TDEP"] = value["TDEP"] / 10

for value in dataframes_dict_605.values():
    value["TDEP"] = value["TDEP"] / 10

#### Converte de polegada para metro

In [10]:
for value in dataframes_dict_551.values():
    value["TDEP"] = value["TDEP"] * 0.0254

for value in dataframes_dict_605.values():
    value["TDEP"] = value["TDEP"] * 0.0254

#### Salva os dataframes como arquivos CSV

In [11]:
for key, value in dataframes_dict_551.items():
    file_name = f"Frames_551/frame_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

for key, value in dataframes_dict_605.items():
    file_name = f"Frames_605/frame_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Frames_551/frame_0.csv criado com sucesso.
Arquivo Frames_551/frame_1.csv criado com sucesso.
Arquivo Frames_551/frame_2.csv criado com sucesso.
Arquivo Frames_551/frame_3.csv criado com sucesso.
Arquivo Frames_551/frame_4.csv criado com sucesso.
Arquivo Frames_551/frame_5.csv criado com sucesso.
Arquivo Frames_551/frame_6.csv criado com sucesso.
Arquivo Frames_551/frame_7.csv criado com sucesso.
Arquivo Frames_551/frame_8.csv criado com sucesso.
Arquivo Frames_605/frame_0.csv criado com sucesso.
Arquivo Frames_605/frame_1.csv criado com sucesso.
Arquivo Frames_605/frame_2.csv criado com sucesso.
Arquivo Frames_605/frame_3.csv criado com sucesso.


#### Remove valores de profundidade que não estão presentes no primeiro frame

In [12]:
for key, value in dataframes_dict_551.items():
    dataframes_dict_551[key] = value[value["TDEP"].isin(dataframes_dict_551[0]["TDEP"])]

for key, value in dataframes_dict_605.items():
    dataframes_dict_605[key] = value[value["TDEP"].isin(dataframes_dict_605[0]["TDEP"])]

#### Junta os 9 dataframes do poço 551 em um dataframe unificado

#### Junta os 4 dataframes do poço 605 em um dataframe unificado

In [13]:
dataframe_unificado_551 = unifica_dataframes(dataframes_dict_551)
dataframe_unificado_605 = unifica_dataframes(dataframes_dict_605)

#### Mantém apenas uma curva de resistividade profunda

In [14]:
dataframe_unificado_filtrado_551 = remove_colunas(dataframe_unificado_551, ['RT', 'AHF90'])

## Cria dataframes para os demais poços

In [15]:
dlis_df_dict = {}   # Conterá os dataframes respectivos aos poços

# Curvas de perfis escolhidas
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'RHOZ', 'DRHO', 'BSZ', 'BS', 'HCAL', 'CAL', 'CALI', 'DCALI', 'DCAL', 'PE', 'DT', 'DTC', 'ILD', 'RILD', 'IEL', 'AIT90', 'AHT90', 'RT90', 'AT90', 'AO90', 'RT', 'AF90', 'AHF90', 'AFH90', 'LLD', 'RLLD', 'HDRS', 'HLLD', 'LL7', 'RLL7']


# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for chave, poco in dli_dict.items():

    # Armazenando as curvas que serão utilizadas em uma lista
    curvas_utilizadas = [
        channel.name                                    # Os elementos da lista serão os nomes das curvas
        for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
        if channel.name in curvas_escolhidas            # As curvas que não utilizaremos não serão armazenadas na lista
    ]
    conjunto_aux = set(curvas_utilizadas)
    curvas_utilizadas_sem_duplicados = list(conjunto_aux)
    
    #frames = [np.atleast_1d(frame) for frame in poco.frames]
    #curvas = np.concatenate(frames, axis=0)
    try:
        dataframe = pd.DataFrame()
        
        for frame in poco.frames:

            curvas = frame.curves()

            dataframe = pd.concat([dataframe, pd.DataFrame(curvas[curvas_utilizadas_sem_duplicados])])

        # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
        dlis_df_dict[chave] = dataframe
    except:
        pass


In [16]:
dlis_df_dict['1-BRSA-605-SE'] = dataframe_unificado_605
dlis_df_dict['1-FSG-1-SE'] = dataframe_unificado_filtrado_551
dlis_df_dict.keys()

dict_keys(['1-FSJQ-1-SE', '1-BRSA-595-SE', '1-BRSA-659-SE', '1-BRSA-605-SE', '1-FSG-1-SE'])

## Transforma os valores -999.25 em nulos

In [17]:
for poco in dlis_df_dict.values():
    poco.replace([-999.25], [None], inplace = True)

## Aplicando os mnemônicos

In [18]:
aplica_mnemonico(dlis_df_dict, ['BS', 'BSZ'], 'BS')
aplica_mnemonico(dlis_df_dict, ['LLD',	'LL7',	'RLLD',	 'RLL7', 'HDRS', 'HLLD', 'ILD',	'RILD',	'IEL',	'AIT90', 'AHT90', 'RT90', 'AT90', 'AO90', 'RT', 'AF90',	'AHF90', 'AFH90'], 'RESD')
aplica_mnemonico(dlis_df_dict, ['RHOB', 'RHOZ'], 'RHOB')
aplica_mnemonico(dlis_df_dict, ['DTC', 'DT'], 'DT')
aplica_mnemonico(dlis_df_dict, ['HCAL', 'CAL', 'CALI'], 'CAL')
aplica_mnemonico(dlis_df_dict, ['DCAL', 'DCALI'], 'DCAL')
aplica_mnemonico(dlis_df_dict, ['DRHO', 'HDRA'], 'DRHO')

## Adiciona coluna DCAL

In [19]:
add_DCAL(dlis_df_dict)

## Preenche os poços com curvas faltando

In [20]:
# Se um dos poços não tiver uma dessas curvas, adicionamos a coluna da curva e mantemos os valores como None
curvas_obrigatorias = ['TDEP', 'BS', 'CAL', 'DCAL', 'GR', 'RESD', 'DT', 'RHOB', 'DRHO', 'NPHI', 'PE']

# Percorre todos os poços
for poco in dlis_df_dict.values():
    # Percorre todas as curvas obrigatórias
    for curva in curvas_obrigatorias:
        # Se o poço não tiver a curva
        if curva not in poco.columns:
            # Adiciona a coluna e os valores dela = None
            poco[curva] = None

## Temos as seguintes curvas 

In [21]:
for key, poco in dlis_df_dict.items():
    curvas = sorted(poco.keys())
    print(f"{key}: {curvas}")

1-FSJQ-1-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-595-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-659-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-605-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-FSG-1-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']


## Remove valores DRHO e DCAL indesejados (Só depois)

In [ ]:
#limita_curva(dlis_df_dict, "DRHO", -0.15, 0.15)
#limita_curva(dlis_df_dict, "DCAL", -2, 2)

## Inverte a ordem das linhas dos dataframes

In [22]:
for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].iloc[::-1]

## Ordena as colunas dos dataframes

In [23]:
# Simon e Vandelli vao definir a ordem de preferencia
ordem_desejada = ['TDEP', 'BS', 'CAL', 'DCAL', 'GR', 'RESD', 'DT', 'RHOB', 'DRHO', 'NPHI', 'PE']

for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].reindex(columns=ordem_desejada)

## Salva os dados dos dataframes em arquivos CSV

In [24]:
for key, value in dlis_df_dict.items():
    file_name = f"Pocos-pre-processados/poco_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Pocos-pre-processados/poco_1-FSJQ-1-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-BRSA-595-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-BRSA-659-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-BRSA-605-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-FSG-1-SE.csv criado com sucesso.
